# subproces学习笔记  
@author: Rui Zhu  
@create time: 2022-11-20  
@cite: 
* https://docs.python.org/zh-cn/3/library/subprocess.html
* https://www.runoob.com/w3cnote/python3-subprocess.html
* [python中的系统交互](https://www.cnblogs.com/yyds/p/7288916.html#:~:text=1.%20subprocess模块中的常用函数%201%20在Python%203.5之后的版本中，官方文档中提倡通过subprocess.run%20%28%29函数替代其他函数来使用subproccess模块的功能；%202%20在Python,%28%29等上面列出的其他函数来使用subprocess模块的功能；%203%20subprocess.run%20%28%29、subprocess.call%20%28%29、subprocess.check_call%20%28%29和subprocess.check_output%20%28%29都是通过对subprocess.Popen的封装来实现的高级函数，因此如果我们需要更复杂功能时，可以通过subprocess.Popen来完成%E3%80%82%20更多项目)

## subprocess简介  
* subprocess库用于创建子进程, 执行系统命令
* subprocess的设计是为了取代os.system()模块
* 执行命令行, 我们关注一下3点
    1. 命令的动作: 除输出结果外的其他动作, 比如文件读写操作
    2. 命令的输出结果
    3. 命令的状态码: 0表示执行成功;1表示失败

### os模块的实现(旧方法, 不推荐使用)

In [1]:
import os
# 方法1
res = os.system('tree -L 1')
print(f"==> os.system()函数, 输出结果, 返回命令的状态码: {res}")

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file
==> os.system()函数, 输出结果, 返回命令的状态码: 0


In [2]:
# 方法2
res = os.popen('tree -L 1')
type(res)  # 该方法不输出状态码, 也不直接输出结果, 而是将输出结果存进os._wrap_close中|

os._wrap_close

In [3]:
# 可以使用read()或write()方法获取输出结果
print(res.read())

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file



## subprocess实现系统命令的调用

In [4]:
import subprocess

### 函数1: run()
* return包含命令结果的CompletedProcess类
* 返回的类中默认只有命令和returncode, 如果需要其他信息, 需要添加参数设置

args: 要执行的shell命令
* 如果参数shell=True, 则可以使用完整的字符串命令, 不然只能是列表的形式

In [5]:
subprocess.run(args=['tree', '-L', '1'])

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file


CompletedProcess(args=['tree', '-L', '1'], returncode=0)

shell: 将命令通过shell执行

In [6]:
subprocess.run(args='tree -L 1', shell=True)

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file


CompletedProcess(args='tree -L 1', returncode=0)

cwd: 指定子进程所在的目录

In [7]:
subprocess.run(args='tree -L 1', shell=True, cwd='./test_for_subprocess/')

.
├── UDS-22130_f160w_sci.fits
├── feedme_galfit_UDS-22130_f160w.txt
├── mask_UDS-22130_f160w.fits
└── psf_3dhst_UDS-22130_f160w.fits

0 directories, 4 files


CompletedProcess(args='tree -L 1', returncode=0)

check: 检查状态码是否为0
* 当check=True, returncode=0, 程序正常执行
* 当check=True, returncode!=0, 抛出CalledProcessError异常

In [8]:
subprocess.run(args='tree -L 1', shell=True, check=True)
# * 试试下面这句
# subprocess.run(args='treed -L 1', shell=True, check=True)

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file


CompletedProcess(args='tree -L 1', returncode=0)

stdout, stderr, universal_newlines: 这3个参数最好同时设置, 用于收集命令的结果或异常
* universal_newlines=True: 使输出结果以字符串的格式保存在类中

In [9]:
res = subprocess.run(
    args='tree -L 1', shell=True, 
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )
res  # stdout=subprocess.PIPE时, 将命令结果存到类中

CompletedProcess(args='tree -L 1', returncode=0, stdout='.\n├── subprocess_tutorial.ipynb\n└── test_for_subprocess\n\n1 directory, 1 file\n', stderr='')

In [10]:
res = subprocess.run(
    args='tree -L 1', shell=True, 
    stdout=True, stderr=subprocess.PIPE, universal_newlines=True
    )
res  # stdout=True时, 输出命令结果

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file


CompletedProcess(args='tree -L 1', returncode=0, stderr='')

timeout: 设置超时时间

In [11]:
subprocess.run(args='tree -L 1', shell=True, timeout=0.01)

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file


CompletedProcess(args='tree -L 1', returncode=0)

### 函数2: Popen()
* Popen()函数时run()函数的底层实现, 因此具有run()的全部功能, 和扩展的功能

In [12]:
popen = subprocess.Popen(
    args='tree -L 1', # 执行的命令
    shell=True, # 在终端中执行
    stdout=subprocess.PIPE, # 执行结果存进类中
    universal_newlines=True  #输出格式为字符串
    )
type(popen)  # 打印命令结果, 返回Popen类


subprocess.Popen

In [13]:
print(popen.stdout.read())

.
├── subprocess_tutorial.ipynb
└── test_for_subprocess

1 directory, 1 file



## 实战
* 在`test_for_subprocess`中run galfit

In [14]:
# 本实例的演示文件
from pathlib import Path
path = Path("./test_for_subprocess/")
origin_files = list(path.iterdir())
print(origin_files)

[PosixPath('test_for_subprocess/.DS_Store'), PosixPath('test_for_subprocess/UDS-22130_f160w_sci.fits'), PosixPath('test_for_subprocess/psf_3dhst_UDS-22130_f160w.fits'), PosixPath('test_for_subprocess/feedme_galfit_UDS-22130_f160w.txt'), PosixPath('test_for_subprocess/mask_UDS-22130_f160w.fits')]


In [15]:
path_feedme = Path("/Users/rui/Code/1_Astronote/20_subprocess/test_for_subprocess/feedme_galfit_UDS-22130_f160w.txt")
cmd = f"galfit {path_feedme}"

res = subprocess.run(
    args=cmd, # cmd为待执行命令
    shell=True, # 在shell中执行
    cwd=path_feedme.parent, # 执行命令的路径
    stdout=subprocess.PIPE, # 以字符串的形式回传结果进类
    stderr=subprocess.PIPE, 
    universal_newlines=True
    )

# 实例结束, 清除生成的文件
for file in list(path.iterdir()):
    if file not in origin_files:
        file.unlink()
    else:
        pass

In [16]:
print(f"输入的命令: {res.args}")
print(f"命令的returncode: {res.returncode}")

输入的命令: galfit /Users/rui/Code/1_Astronote/20_subprocess/test_for_subprocess/feedme_galfit_UDS-22130_f160w.txt
命令的returncode: 0


In [17]:
print(f"命令的打印结果: {res.stdout}")

命令的打印结果: 
GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: /Users/rui/Code/1_Astronote/20_subprocess/test_for_subprocess/feedme_galfit_UDS-22130_f160w.txt


# IMAGE and GALFIT CONTROL PARAMETERS
A) UDS-22130_f160w_sci.fits      # Input data image (FITS file)
B) galfit_result_UDS-22130_f160w.fits      # Output data image block
C) none                # Sigma image name (made from data if blank or "none") 
D) psf_3dhst_UDS-22130_f160w.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) mask_UDS-22130_f160w.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) none                # File with parameter constraints (ASCII file) 
H) 1    99   1    99   # Image region to fit (xmin xmax ymin ymax)
I) 100    100          # Size of the convolution box (x y)
J) 26.946              # Magnitude photometric zeropoint 
K) 0.060  0.060        # Plate scale (dx dy)   [arcsec per pixel]
O) regular    

In [18]:
# 将执行的结果输出到txt文件
dir_download = Path("/Users/rui/Downloads")
with open(dir_download / 'log.txt', 'w') as f:
    f.write(res.stdout)
    f.close()